In [1]:
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.svm import SVC
import models

In [2]:
data = pd.read_csv("data.csv", quotechar='"')
X = data.text
y = data[["term1", "term2", "term3"]]

yvalues = y.as_matrix()

final = []
for i in range(0, len(yvalues)):
    r = yvalues[:][i:i + 1]
    r = np.array(r[[~pd.isnull(r)]])
    final.append(r)

final = np.array(final)

In [3]:
mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(final)

In [4]:
cv = CountVectorizer(stop_words="english")

In [20]:
# Train Test Split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=3)

In [21]:
X_train_dtm = cv.fit_transform(X_train)
X_test_dtm = cv.transform(X_test)

In [22]:
model = models.UniformOVA(c=1, t1=0.1, t2=0.1)
model.fit(X_train_dtm, Y_train)
Y_pred_class = model.predict(X_test_dtm)

score = metrics.f1_score(Y_test, Y_pred_class, average='samples')
print(score)

0.414469453376


In [8]:
from sklearn.model_selection import GridSearchCV
t1_range = np.arange(0.1, 1.1, 0.1)
t2_range = np.arange(0.1, 1.1, 0.1)
param_grid = dict(t1=t1_range, t2=t2_range)
model = models.UniformOVA()
grid = GridSearchCV(model, param_grid, cv=10, scoring="f1_samples")
%time grid.fit(X_train_dtm, Y_train)

Wall time: 33min 54s


GridSearchCV(cv=10, error_score='raise',
       estimator=UniformOVA(c=1, null_dv=-99, t1=0, t2=0), fit_params={},
       iid=True, n_jobs=1,
       param_grid={'t1': array([ 0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9,  1. ]), 't2': array([ 0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9,  1. ])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='f1_samples', verbose=0)

In [9]:
# best score / params
print(grid.best_score_)
print(grid.best_params_)

0.443137131686
{'t1': 0.10000000000000001, 't2': 0.10000000000000001}
